# Pika-analyysi Exceliin

Frekvenssitaulukot, ristiintaulukoinnit, keskeiset tunnusluvut ja korrelaatiot Excel-tiedostoon.

Pika-analyysiä voit käyttää seuraavasti:

* Vaihda avattavan datan tilalle oma data
* Suorita koodi
* Koodin suorittamisen jälkeen Excel-tiedosto **pika.xlsx** löytyy output-kansiosta.

In [1]:
import pandas as pd

# Jos muuttujalla on ainutkertaisia arvoja yli RAJAn, niin muuttuja tulkitaan määrälliseksi
# RAJA-arvoa voit tarvittaessa muuttaa
RAJA = 10

# Vaihda tähän analysoitava data
df = pd.read_excel('https://taanila.fi/titanic.xlsx')

In [2]:
# Alustavat toimet

# luodaan Listat määrällisistä ja kategorisista muuttujista
kvantit = []
kategoriset = []
for muuttuja in df:
    if (len(df[muuttuja].unique())>RAJA) & (df[muuttuja].dtype in ['int64', 'float64']):
        kvantit.append(muuttuja)
    elif len(df[muuttuja].unique())<=RAJA:
        kategoriset.append(muuttuja)

# Luon ExcelWriter-olion, joka kirjoittaa pika.xlsx-tiedostoon
writer = pd.ExcelWriter('output/pika.xlsx', engine='xlsxwriter')

# Prosentti-muotoilu, jota käytän frekvenssitaulukoissa ja ristiintaulukoinneissa
format = writer.book.add_format({'num_format':'0.0 %'})


# Frekvenssitaulukot

if kategoriset:
    # Pidän kirjaa Excelin rivinumerosta rivi-muuttujan avulla
    rivi = 0
    # Käyn for-silmukalla läpi kaikki kategoriset muuttujat
    for muuttuja in kategoriset:
        # Lasken frekvenssit df1-nimiseen dataframeen
        df1 = pd.crosstab(df[muuttuja], 'f')
        # Lasken df1:een prosentit
        df1['%'] = df1/df1.sum()
        # Lisään df1:een Yhteensä-rivin
        df1.loc['yhteensä'] = df1.sum()
        # Kirjoitan frekvenssitaulukon Excel-tiedoston Frekvenssit-taulukkovälilehdelle
        df1.to_excel(writer, sheet_name='Frekvenssit', startrow=rivi)
        # Kasvatan rivinumeroa; shape[0] antaa df1:n rivimäärän
        rivi = rivi + df1.shape[0] + 2
    # Lisään C-sarakkeeseen prosenttimuotoilun
    writer.sheets['Frekvenssit'].set_column('C:C', cell_format=format)


# Ristiintaulukoinnit
    
if len(kategoriset) > 1:
    rivi = 0
    for muuttuja1 in kategoriset:
        for muuttuja2 in kategoriset:
            if muuttuja1 != muuttuja2:
                df1 = pd.crosstab(df[muuttuja1], df[muuttuja2])
                df2 = pd.crosstab(df[muuttuja1], df[muuttuja2], normalize='columns')
                df2.index.name=muuttuja1+'/'+muuttuja2 
                df2.loc['n'] = df1.sum()
                df2.to_excel(writer, sheet_name='Ristiintaulukoinnit', startrow=rivi)
                for i in range(rivi+1, rivi+df2.shape[0]):
                    writer.sheets['Ristiintaulukoinnit'].set_row(i, cell_format=format)
                rivi = rivi+df2.shape[0]+2
        

# Tunnusluvut
            
if kvantit:
    df1 = df[kvantit].describe()
    df1.to_excel(writer, sheet_name='Tunnusluvut')
    # Tunnusluvut kategoristen määräämissä ryhmissä
    if kategoriset:
        rivi = df1.shape[0]+2
        for muuttuja1 in kategoriset:
            for muuttuja2 in kvantit:
                if muuttuja1 != muuttuja2:
                    df1 = df.groupby(muuttuja1)[muuttuja2].describe()
                    df1.index.name = muuttuja1+'/'+muuttuja2
                    df1.to_excel(writer, sheet_name='Tunnusluvut', startrow = rivi)
                    rivi = rivi + df1.shape[0]+2


# Korrelaatiot                
                
if kvantit:
    df1 = df[kvantit].corr()
    df1.to_excel(writer, sheet_name='Korrelaatiot')


# Excel-tiedoston tallennus
writer.close()

Taulukon otsikoiden muotoilu Excelissä on konstikasta. Katso https://xlsxwriter.readthedocs.io/working_with_pandas.html#formatting-of-the-dataframe-headers

Lisätietoa Pythonin käytöstä data-analytiikassa https://tilastoapu.wordpress.com/python/